<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_8_tuning_healthnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    986      0 --:--:-- --:--:-- --:--:--   982
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
Unpacking objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-8.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-8.validation.csv",header=None)
valid.head()

,0,1
0,0,scared to death
1,2,heel pain
2,5,increased heart rate
3,6,stage 1 diastolic dysfunction
4,8,arthritis


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-8.test.csv",header=None)
test.head()

,0,1
0,1,Abdominal rash
1,5,increase my heart rate
2,5,tachycardia
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_healthnews_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [18]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.975612,5.791724,0.144085,02:04


In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.805943,5.252346,0.188951,02:04


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.424060,5.198531,0.194252,02:05


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.368093,5.151604,0.199609,02:04


In [0]:
learn.save_encoder('healthnews_fold8_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('healthnews_fold8_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.677664,4.431220,0.307028,01:10


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.965913,4.055123,0.325524,01:21


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.578943,3.731658,0.364982,03:25


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.357512,3.472690,0.401973,03:30
1,3.112597,3.375459,0.429100,03:26


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.022678,3.258812,0.445129,03:30
1,2.897980,3.199243,0.443896,03:18


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.793795,3.196810,0.458693,03:22
1,2.649914,3.073516,0.473490,03:25


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.581128,3.048414,0.471023,03:20
1,2.554214,2.988823,0.475956,03:23


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.548104,3.031721,0.480888,03:26
1,2.326076,2.940617,0.493218,03:37


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.428730,2.973768,0.475956,03:29
1,2.427078,2.958995,0.490752,03:10
2,2.355523,2.949504,0.491985,03:29
3,2.174817,2.858644,0.504316,03:16


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.179496,2.903793,0.482121,03:24
1,2.275693,2.901320,0.498150,03:19
2,2.240101,2.913735,0.509248,03:29
3,2.134108,2.902750,0.515413,03:24


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.156827,2.936579,0.509248,03:17
1,2.241539,2.954949,0.496917,03:21
2,2.175978,2.885198,0.512947,03:28
3,2.113139,2.909708,0.509248,03:19


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.052468,2.923590,0.511714,03:15
1,2.155013,3.007140,0.508015,03:15
2,2.114669,3.005078,0.504316,03:21
3,1.956633,2.974271,0.505549,03:24


In [0]:
# save the best model

learn.save_encoder('ask-8-tuning-healthnews')

In [43]:
learn.load_encoder('ask-8-tuning-healthnews')
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.979246,2.982267,0.510481,01:10
1,1.976293,2.975302,0.509248,01:07
2,2.095936,3.022153,0.503083,01:09
3,2.005729,2.990869,0.504316,01:07


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.011235,3.050959,0.504316,01:11
1,2.024074,2.984217,0.506782,01:09
2,2.051714,3.063250,0.509248,01:06
3,2.016510,2.978414,0.512947,01:09


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.035568,3.018520,0.515413,01:05
1,2.033511,2.982514,0.512947,01:04
2,2.019748,3.040243,0.504316,01:11
3,2.112965,3.174741,0.503083,01:06


In [46]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.891448,2.956547,0.510481,01:12
1,2.090882,3.117831,0.503083,01:05
2,1.998415,3.026412,0.501850,01:10
3,1.958200,3.037772,0.504316,01:06


In [47]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.009102,3.086154,0.498150,01:11
1,2.062963,3.027112,0.509248,01:11
2,2.006564,3.081214,0.504316,01:07
3,2.004483,3.065769,0.500617,01:06


# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[528, 258, 949, 8, 8, 8, 949, 20, 403, 197, 491, 491, 31, 31, 1035, 736, 635, 487, 37, 289, 896, 896, 1035, 1028, 326, 265, 325, 1035, 775, 1028, 1028, 228, 1028, 1028, 1028, 1028, 933, 92, 92, 228, 541, 289, 289, 228, 228, 1028, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 686, 1028, 1028, 80, 80, 80, 80, 85, 1028, 228, 1028, 1028, 228, 228, 94, 94, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 289, 455, 713, 289, 1035, 492, 1028, 100, 100, 265, 265, 265, 100, 100, 994, 193, 108, 108, 289, 1035, 289, 182, 289, 289, 289, 289, 289, 289, 111, 289, 1028, 117, 745, 778, 125, 125, 125, 1028, 1028, 125, 125, 125, 125, 778, 289, 386, 343, 355, 1035, 360, 289, 330, 137, 137, 473, 142, 847, 151, 151, 1028, 157, 1035, 1002, 165, 1028, 1028, 210, 172, 289, 289, 289, 1028, 245, 1028, 102, 182, 182, 182, 182, 289, 289, 183, 183, 183, 1

In [50]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

208
0.24018475750577367
